In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [80]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [81]:
# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price aok-align-center'}).find("span" , attrs={'class':'a-offscreen'}).text

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-price aok-align-center'}).find("span" , attrs={'class':'a-offscreen'}).text
        except:
            price = ""

    return price

In [82]:
def get_rating(soup):

    try:
        rating = soup.find("span", attrs={'class':'a-icon-alt'}).text

    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

In [83]:
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).text

        
    except AttributeError:
        review_count = ""

    return review_count

In [84]:
#Function to extract Product Description
def get_description(soup):

    try:
        desc_list = soup.find("div", attrs={'id':'feature-bullets'}).find_all("span",attrs={'class':'a-list-item'})

        desc=""
        for point in desc_list:
            desc=point.text+desc
    except AttributeError:
        desc=""
    
    return desc;
    

In [122]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.42', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=bags&page=20&crid=TM4YWVYRN5KW&qid=1685029914&sprefix=bag%2Caps%2C262&ref=sr_pg_20"
#     
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")


    # find all anchor tags with product link
    links = soup.find_all("a","a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")

    # Store the links
    links_list = []
    
#     print(links)

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))
        
#     print(links_list)

    d = {"title":[], "price":[], "rating":[], "numberofreviews":[], "description":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['numberofreviews'].append(get_review_count(new_soup))
        d['description'].append(get_description(new_soup))

        
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [99]:
amazon_df

,title,price,rating,numberofreviews,description
0,NIVIA Beast Gym Bag-4 Polyester/Unisex Gym Bag...,₹409.00,4.2 out of 5 stars,"3,290 ratings",Durable zippers and enhanced by strong stitch...
4,"Lenovo 15.6"" Professional Backpack, Made in In...",₹700.00,4.2 out of 5 stars,755 ratings,"Styled in classic dark blue and dark grey, th..."
9,ADISA 32L Large Laptop Backpack Office Bag Col...,₹599.00,3.8 out of 5 stars,141 ratings,Reflector Logo / Adjustable Padded Shoulder S...
14,"Lenovo 15.6"" Casual Backpack B210",₹774.00,4.2 out of 5 stars,"26,983 ratings",Country of Origin: China In the Box : 15.6-...


In [119]:
d

{'title': [],
 'price': [],
 'rating': [],
 'numberofreviews': [],
 'description': []}